# Affective Computing - Exercise 2




#### Objective

The objective of the exercise is to build a facial expression recognition system. The system includes face preprocessing, feature extraction and classification. In the exercise, you will learn how to preprocess a facial expression image, extract features from an image or a video, and classify a video into a category.

Specifically, the region of interest (i.e., facial image) is extracted using face tracking, face registration and face crop functions.  Basic spatiotemporal features (i.e., LBP-TOP features) are extracted using LBP-TOP. For classificaiton of the extracted features Support Vector Machine (SVM) classifiers are trained. 50 videos from 5 participants are used to train the emotion recognition, using spatiotemporal features. The rest of the data (50 videos) are used to evaluate the performances of the trained recognition system.

#### Database
The original facial expression data is a sub-set of eNTERFACE (acted facial expression), from ten actors acting **happy** and **sadness** behaviors. The used dataset in the exercise includes 100 facial expression samples. 

<!---#### Help-->
<!--- The data and toolbox files used in this exercise can be found in the Affective Computing course webpage (see the Noppa system).
-->

<!---In the exercise, you should know that some basic python libries (numpy, scikit-image, scipy, pyploy, sklearn) before the programming. If you have questions, send your question to us: 
* [Henglin.Shi@oulu.fi](mailto:Henglin.Shi@oulu.fi)
* [Yante.Li@oulu.fi](mail.Yante.Li@oulu.fi)-->

<!---Use the following website to help the usage of python libries. 
* numpy: http://www.numpy.org/
* skimage: https://scikit-image.org/
* scipy: https://www.scipy.org/
* matplotlib.pyplot: https://matplotlib.org/api/pyplot_api.html
* sklearn: http://scikit-learn.org/stable/
    * sklearn.svm: http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
    * skealrn.metrics.confusion_matrix: http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html-->

## Task 1. Face preprocessing

In this part, you are supposed to process a face image. There are three subtasks you need to do:
* **Task 1.1.** Detect face and facial landmarks using the [`DLib`](http://dlib.net/) library.
* **Task 1.2.** You are asked to perform a face registration task using a set of fixed landmarks from a standard model, and extract face from the registered image.
* **Task 1.3.** Visualize your result using subplots.

### Task 1.1. Extract facial landmarks <font color='red'>(1 point)</font>
**Steps**:
1. Load the example image *example_img.jpg*, using the [`plt.imread()`](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imread.html) function for example.
2. As a sanity check visualize the image to see what you are working with.
3. Initialize a face detector and a face landmarks detector. We have provide the code of this part, please learn to use them.
4. Detect the face
5. Detect the face landmarks
6. Transfrom the detected result to a 2-D numpy array by using the provided `shape2points` function from the *face_lib.py*.
7. Visualize the landmarks with the image

*Here is an example for facial landmarks extraction: http://dlib.net/face_landmark_detection.py.html (Note that this examples assumes multiple faces in the image. The images here only have a single sample, and no loop is required.)*



In [9]:
# Loading required libraries
import dlib
import numpy as np
import matplotlib.pyplot as plt
from face_lib import shape2points

ModuleNotFoundError: No module named 'dlib'

In [ ]:
# 1. Load example_img.jpg, using plt.imread()
example_img = plt.imread("example_img.jpg")
# 2. Visualize the image using plt.imshow()
plt.figure(figsize=(8, 8))
plt.imshow(example_img)
plt.title("Original Image")
plt.show()

# 3. Initializing face detector and shape predictor
detector = dlib.get_frontal_face_detector()
#the shape predictor is a neural network that is loaded from the .dat file
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
# 4. Detect face, return rectangles, each rectangle corresponds to one face.
dets = detector(example_img, 1)
# 5. Extract the shape of the face in the first rectangle (using the first element of the rectangles variable)
shape = predictor(example_img, dets[0])
# 6. Extract facial landmarks from shape by calling the shape2points() function.
keypoints = shape2points(shape)

# 7. Visualize the landmarks (keypoints) by first plotting the image and then using plt.scatter() for the landmarks
plt.figure(figsize=(8, 8))
plt.imshow(example_img)
plt.scatter(keypoints[:, 0], keypoints[:, 1], c="red", s=6)
plt.title("Image with Facial Landmarks")
plt.show()

### Task 1.2. Face normalization <font color='red'>(1 point)</font>
**Steps**:
1. Load the landmark position of a standard face model. We provide these positions in a csv file, and also the code block to read these positions.
2. Calculate the transformation between your detected landmarks position and the standard face model landmark positions using the [`skimage.transform.PolynomialTransform()`](http://scikit-image.org/docs/dev/api/skimage.transform.html#skimage.transform.PolynomialTransform) class and its *estimate()* methods. 
    1. Instantiate a PolynomialTransform object by calling transform.PolynomialTransform()
    2. Call its estimate() method to calculate the transformation between the two sets of points. The manual of this method can be found in the same page which introduced of this class. 
3. Transform the example image using the calculated transformation to register (map) the example image into a space of the standard face model. You can use the [`skimage.transform.warp()`](http://scikit-image.org/docs/dev/api/skimage.transform.html#skimage.transform.warp) function to perform this.
4. Crop the face from the registered face using the standard face model landmarks. The cropping function is provided in the *face_lib.py*, you can directly use it after importing. 
5. Also extract the face from the example image using your detected landmarks.
<!---Use **cropFace** to crop the facial image from the background. For details on the **cropFace** function, please read the function in **exercise1Lib.py**. Usually, we have to do the face registration to remove the subject characteristics. To do a  face registration, firstly you need a set of landmarks of a standard face model. Then you calculate the transformation between you detected landmarks and the standard landmarks. Last you apply the transformation on your example image/-->

In [ ]:
import scipy.io as sio

# Loading data using scipy.io.loadmat(), or sio.loadmat
mdata = sio.loadmat("Task3_data.mat")

# Load 'training_data'
sample_train = mdata["training_data"]

# Load 'testing_data'
sample_test = mdata["testing_data"]

# Load 'training_class'
label_train = mdata["training_class"]

# Load 'testing_class'
label_test = mdata["testing_class"]

#Print the shapes from each of the data splits
print(f"Shape of training_data: {sample_train.shape}")
print(f"Shape of testing_data: {sample_test.shape}")
print(f"Shape of training_class: {label_train.shape}")
print(f"Shape of testing_class: {label_test.shape}")

<!---**Then you need to calculate the transformation using [`skimage.transform.PolynomialTransform()`](http://scikit-image.org/docs/dev/api/skimage.transform.html#skimage.transform.PolynomialTransform)**. This class has several methods, expecially, you need to use the **estimate()** method to calculate the transformation between two sets of points. **Registering the example face** by transforming the example image into the standard model space, using [`skimage.transform.warp()`](http://scikit-image.org/docs/dev/api/skimage.transform.html#skimage.transform.warp) and you calculated transformation. **Cropping the original example image according to the detected landmarks, and the registered image according to the standard face mode. **-->

In [ ]:
from skimage import transform
from face_lib import crop_face

# 2. Calculating the transformation between the two set of points 
# 2.1 Instantiating a PolynomialTransform() transform function
tform = transform.PolynomialTransform()

# 2.2 Calculating the transformation by calling the estimate() method.
tform.estimate(standard_model, keypoints)

# 3. Warp the example image using the transform.warp() function
registered_face_image = transform.warp(example_img, tform, output_shape=(example_img.shape[0], example_img.shape[1]))

# 4. Crop the face from registered image using the provided crop_face function.
cropped_registered_face = crop_face(registered_face_image, standard_model)

# 5. Cropping the face from the example image using detected landmarks.
cropped_example_face = crop_face(example_img, keypoints)

### Task 1.3. Display result <font color='red'>(2 points)</font>
Here you are asked to draw a figure with 3 x 2 subplots using [`matplotlib.pyplot.subplots()`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.subplots.html). Please read the manual of it and also the [`matplotlib.pyplot`](https://matplotlib.org/api/pyplot_api.html). 

Each of the subplots should have the following images:
* subplot [0, 0]: the original example image and detected landmarks.
* subplot [1, 0]: the face cropped from the example image. 
* subplot [2, 0]: the histogram of the face cropped from the example. 

*As an example, the three subplots are given above.* Then you need to implement:
* subplot [0, 1]: the registered face image.
* subplot [1, 1]: the face cropped from the registered face image.
* subplot [2, 1]: the histogram of the face cropped from the registered face image.



In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from skimage import img_as_ubyte

# Constructing figure with 2x3 subplots
fig, ax = plt.subplots(3, 2, figsize=(15, 15))

# subplot [0,0]: show the original example image
ax[0, 0].imshow(example_img)
ax[0, 0].set_title("Original Image with Landmarks")

# Placing detected landmarks on subplot [0,0], we provide an exmaple to do this.
ax[0, 0].scatter(keypoints[:, 0], keypoints[:, 1], c="red", s=6)

    
# subplot [1,0]: show the face cropped from the example image.
ax[1, 0].imshow(cropped_example_face, "gray")
ax[1, 0].set_title("Cropped Example Face")

# subplot [2,0]: show the histogram of the face cropped from the example image.
ax[2, 0].hist(cropped_example_face.ravel(), bins=256)
ax[2, 0].set_title("Histogram of Cropped Example Face")

# subplot [0,1]: show the registered image
ax[0, 1].imshow(registered_face_image)
ax[0, 1].set_title("Registered Face Image")

# place the model landmarks on the registered image   
ax[0, 1].scatter(standard_model[:, 0], standard_model[:, 1], c="blue", s=6)

# subplot [1,1]: show the face cropped from the registered image
ax[1, 1].imshow(cropped_registered_face, "gray")
ax[1, 1].set_title("Cropped Registered Face")

# subplot [2,1]: show the histogram of the face cropped from the registered image.
ax[2, 1].hist(cropped_registered_face.ravel(), bins=256)
ax[2, 1].set_title("Histogram of Cropped Registered Face")

fig.tight_layout()
plt.show()

### Question 1. What is the difference between the cropped example image (subplot [1, 0]) and the cropped registered image (subplot [1, 1])? What is the mapping between the images called?
*Hints: Look at the two images very closely, the change is very subtle. The code may also provide hints for the transformation name.*

### Your answer:
The cropped example image (subplot [1, 0]) shows the face as it was detected in the original image, potentially with some rotation or scaling depending on the subject's pose. The cropped registered image (subplot [1, 1]) shows the face after it has been geometrically transformed (normalized) to align with a standard face model. This means the registered face is typically frontalized and scaled to a consistent size and orientation.

The mapping between the images is called **face registration** or **face alignment**.

### Question 2. What is the purpose of the operation asked in the previous question, why is it done?
### Your answer:
The purpose of face registration (or face alignment) is to normalize the facial images by removing variations caused by head pose, scale, and rotation. This is done to ensure that all faces are presented in a consistent, standardized manner, which is crucial for subsequent processing steps like feature extraction and classification. By aligning faces to a common reference, the system can focus on extracting features related to expression rather than being confounded by geometric variations. This improves the robustness and accuracy of facial expression recognition systems.

## Task 2. Feature extraction <font color='red'>(2 points)</font>
Here you are asked to extract LBP (Local Binary Pattern) features. LBP has been widely used in face recognition, facial expression recognition and texture classification.
Here you will use the [`skimage.feature.local_binary_pattern()`](http://scikit-image.org/docs/dev/api/skimage.feature.html#skimage.feature.local_binary_pattern) function to extract the LBP features. Steps are explained below.

**Steps**:
1. Define the LBP parameters. Before doing this, carefully read the documentation.
    1.  P: Number of neighbours, set P = 8
    2.  R: Radius of circle, set R = 1.0
    3.  LBP methods: set it as 'nri_uniform'

2. Extract the LBP face by calling the *skimage.feature.local_binary_pattern()* function

3. Calculate the histogram of the LBP face
    1. Caculate the histogram of the LBP face
    2. Normalize the histogram to make the histogram's sum one one (dividing each element by the sum of the histogram).

4. Visualize the result using two subplots.
    1. Draw the LBP face on the left side. Use the *cmap="gray"* parameter
    2. Draw the normalized histogram on the right side, but using *.stem()* function rather *.plot()* for this time.

In [ ]:
from skimage.feature import local_binary_pattern

# 1. Define the parameters to extract LBP features in (8, 1) neighborhood:
#    1.1 Set the number of neighbour P = 8
P = 8
#    1.2 Set the radius if circir R = 1.0
R = 1.0
#    2.3 Set the method as "nri_uniform"
METHOD = 'nri_uniform'

# 2. Extract the LBP face using local_binary_pattern() with the defined parameters
lbp_face = local_binary_pattern(cropped_example_face, P, R, method=METHOD)

# 3. Calculate the histogram of the LBP face (58 bins). Normalize the histogram's sum to one
n_bins = int(lbp_face.max() + 1)
hist, _ = np.histogram(lbp_face, density=True, bins=n_bins, range=(0, n_bins))
hist = hist / hist.sum() # Normalize to sum one

# 4. Visualize your result.
fig, (ax_img, ax_hist) = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
plt.gray()
ax_img.imshow(lbp_face)
ax_img.set_title('LBP image')
ax_img.axis('off')
ax_hist.stem(hist, use_line_collection=True)
ax_hist.set_title('Histogram of LBP values')
ax_hist.set_ylabel('Percentage')
ax_hist.set_xlabel('LBP value')
plt.show()

### Question 3. Why is feature extraction (e.g. LBP) used, why can't we simply classify the (raw) images in their original domain?

### Your answer:
Feature extraction (like LBP) is used because raw image data often contains a lot of redundant information and high dimensionality, which can make classification computationally expensive and prone to overfitting. Raw pixel values are also highly sensitive to variations in lighting, pose, and scale, making direct classification challenging.

Feature extraction transforms the raw data into a lower-dimensional, more discriminative representation that captures essential characteristics (e.g., texture, edges) while being more robust to noise and variations. This makes the classification task more efficient and effective.

## Task 3. Feature Classification

For feature classification the SVM (Support Vector Machine) will be utilized. See the documentation for [`sklearn.svm.SVC()`](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html). Mainly you will use its two methods: **fit()** to training the classifier and **predict()** to use the classifer for classification. There are following three subtasks you need to complete:
* Task 3.1. Load data
* Task 3.2. Train classifiers
* Task 3.3. Evaluate classifiers



### Task 3.1. Load data <font color='red'>(1.5 points)</font>
Firstly, you need to read *.mat* files using python. You can use the [`scipy.io.loadmat()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.io.loadmat.html#scipy.io.loadmat) function to read *.mat* file. In the provided **Task3_data.mat** file, different data are packed by different dictionaries which are list below:
* training_data
* testing_data
* training_class
* testing_class

For example if you would like to load the *training_data* from the file, you can use **scipy.io.loadmat('Task3_data.mat')['training_data']**

Print the shapes of data for a sanity check.



In [ ]:
import scipy.io as sio

# Loading data using scipy.io.loadmat(), or sio.loadmat
mdata = sio.loadmat("Task3_data.mat")

# Load 'training_data'
sample_train = mdata["training_data"]

# Load 'testing_data'
sample_test = mdata["testing_data"]


# Load 'training_class'
label_train = mdata["training_class"]

# Load 'testing_class'
label_test = mdata["testing_class"]

#Print the shapes from each of the data splits
print(f"Shape of training_data: {sample_train.shape}")
print(f"Shape of testing_data: {sample_test.shape}")
print(f"Shape of training_class: {label_train.shape}")
print(f"Shape of testing_class: {label_test.shape}"

### Question 4. The training data (*sample_train*) has a shape of $(50, 708)$. What do the numbers refer to? What is 50 and what is 708?

### Your answer:
The training data (`sample_train`) has a shape of `(50, 708)`. The number `50` refers to the number of samples (e.g., videos or images) in the training dataset. The number `708` refers to the number of features extracted for each sample. In this context, `708` is the dimensionality of the LBP-TOP feature vector for each facial expression sample.

### Task 3.3. Train SVM classifiers <font color='red'>(0.5 points)</font>
Use the **sklearn.svm** library to train Support Vector Machine (SVM) classifiers. The *sample_train* and *sample_test* matrices contain the calculated LBP-TOP features for the training and testing sets, respectively. The block size for LBP-TOP used for training and testing data are 2x2x1. The *label_test* group vector contains the class of samples: 1 = happy, 2 = sadness, corresponding to the rows of the training data matrices.

**Steps**:
1.  Construct an SVM classifier object using a linear kernel. See [`sklearn.svm.SVC()`](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC).

2.  Use the [`fit()`](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC.fit) method and the *sample_train* and *label_train* to train your classifier.

In [ ]:
from sklearn import svm
# 1. Initializing an SVM classifier, using linear kernel
classifier = svm.SVC(kernel='linear')

# 2. Use the classifier to fit your training data
classifier.fit(sample_train, label_train.ravel())

### Task 3.3. Evaluate your classifiers <font color='red'>(2 points)</font>
**Steps**:
1. Use your trained classifer to classify the *sample_train* and *sample_test*, using the [`predict()`](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC.predict) method.
2. Calculate the classification accuracies when classifying the *sample_train* and *sample_test*, respectively. The correct class labels corresponding with the rows of the training and testing data matrices are in the variables *label_train* and *label_test*, respectively.
3. Calculate the confusion matrices when evaluating both the dataset by using [`sklearn.metrics.confusion_matrix()`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html).


In [ ]:
# 1. Predict the training data and testing data.
pred_train = classifier.predict(sample_train)
pred_test = classifier.predict(sample_test)

# 2. Calculate the accuracies of your prediction on training data and testing data, respectively.
#    2.1 Calculate the accuracy when classifying the training data 
accuracy_train = np.sum(pred_train == label_train.ravel()) / len(label_train)
print(f"Training data accuracy: {accuracy_train:.2f}")

#    2.2 Calculate the accuracy when classifying the test data
accuracy_test = np.sum(pred_test == label_test.ravel()) / len(label_test)
print(f"Testing data accuracy: {accuracy_test:.2f}")

from sklearn.metrics import confusion_matrix

# 3. Print/Draw the confusion matrix using sklearn.metrics.confusion_matrix
#    3.1 Calculate the confusion matrix when classifying the training data
conf_matrix_train = confusion_matrix(label_train, pred_train)
print("Training data confusion matrix: ")
print(conf_matrix_train)

#    3.2 Calculate the confusion matrix when classifying the testing data
conf_matrix_test = confusion_matrix(label_test, pred_test)
print("Testing data confusion matrix: ")
print(conf_matrix_test)

### Question 5. Compare the classification results from train data and test data. Which one achieves a better accuracy? Why?

### Your answer:
Typically, the training data accuracy is better than the test data accuracy. This is because the classifier is trained on the training data and optimizes its parameters to perform well on it. The test data, however, is unseen data, and its accuracy reflects how well the model generalizes to new, unobserved samples. A significant drop in accuracy from training to testing data can indicate overfitting, where the model has learned the training data too well, including its noise, and struggles with new data.

### Question 6. Look at the confusion matrix of test data. In which class did the classifier perform better, the happiness or sadness?

### Your answer:
To determine which class (happiness or sadness) the classifier performed better in, we need to look at the confusion matrix for the test data (`conf_matrix_test`). The confusion matrix is structured as follows:

```
[[True Negatives (TN)  False Positives (FP)]
 [False Negatives (FN) True Positives (TP)]]
```

Assuming class 1 is 'happy' and class 2 is 'sadness':
- **Accuracy for Happy (Class 1)**: `TN / (TN + FP)` (correctly classified happy samples out of all actual happy samples)
- **Accuracy for Sadness (Class 2)**: `TP / (TP + FN)` (correctly classified sadness samples out of all actual sadness samples)

Without the actual output of `conf_matrix_test`, I cannot give a definitive answer. However, by examining the values in the confusion matrix, one can compare the diagonal elements (correct classifications) relative to the off-diagonal elements (misclassifications) for each class to determine which emotion was better recognized.